In [2]:
import numpy as np
import pandas as pd
from math import exp

In [62]:
# sure working functions
def sigmoid(value):
    return 1.0/(1+exp(value * (-1)))

# output = float
def sigma(arr_weight, arr_input, bias=0):
    #len(arr_weight) = len(arr_input)
    return arr_weight.dot(arr_input.transpose())[0] + bias

# hidden_layer = int (number of hidden layers)
# nb_nodes = arr[int] (number of nodes per hidden layer)
# len_input_matrix = int (number of features)
# Output: List of Matrixes
# Method: He initialization
# Link: https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78
def initialize_weights(hidden_layer, nb_nodes, len_input_matrix):
    nb_nodes = nb_nodes.astype(int)
    arr_weights = list()
    for i in range(hidden_layer):
        if i==0:
            nb_nodes_prev = len_input_matrix
        else:
            nb_nodes_prev = nb_nodes[i-1]
        weight_matrix = np.random.randn(nb_nodes[i], nb_nodes_prev) * np.sqrt(2/(nb_nodes_prev+nb_nodes[i]))
        arr_weights.append(weight_matrix)
    return arr_weights

In [57]:
# functions in progress
def error(feed_forward_output, target_output):
    return 0.5*((target_output-feed_forward_output)**2)

def propagate_error_output_layer(feed_forward_output, target_output):
    return feed_forward_output*(1-feed_forward_output)*(target_output-feed_forward_output)

def propagate_error_hidden_layer_neuron(arr_weight_input, arr_neuron_input, arr_weight_output, arr_neuron_output):
    #Input here means input in a single neuron, while output means output of a single neuron
    #len(arr_weight) = len(arr_input)
    sigma_input, sigma_output = sigma(arr_weight_input, arr_neuron_input), sigma(arr_weight_output, arr_neuron_output)
    return sigmoid(sigma_input) * (1 - sigmoid(sigma_input)) * sigma_output

def update_weight_neuron(weight_prev_prev, weight_prev, nu, alfa, error, output_neuron):
    # weight_prev_prev = previous of weight_prev
    return weight_prev + weight_prev_prev * alfa + nu*error*output_neuron

In [69]:
# input_matrix = matrix[float] (data) (asumsi, kolom terakhir adalah hasil klasifikasi)
# hidden_layers = int (number of hidden layers)
# nb_nodes = arr[int] (number of nodes per hidden layer)
# nu = float (momentum)
# alfa = float (learning rate)
# epoch = int (number of training loops)
# batch_size = int (mini-batch)
# output = FFNN prediction model (list of matrix)
def mini_batch_gradient_descent(input_matrix, hidden_layer, nb_nodes, nu, alfa, epoch, batch_size=1):
    
    #transpose-slicing, memisah input dan label
    col_width = input_matrix.shape[1]
    input_data = (input_matrix.transpose()[0:col_width-1]).transpose()
    label_data = (input_matrix.transpose()[col_width-1:col_width]).transpose()
    #print(input_data, "\n")
    #print(label_data, "\n")
    
    # add output layer, dimana node terakhir adalah 1
    hidden_layer = hidden_layer + 1
    nb_nodes = np.append(nb_nodes, [1])
    
    FFNN = initialize_weights(hidden_layer, nb_nodes, col_width-1)
    
    for i in range(epoch):
        FFNN_prev = FFNN
    
    output = 0 #dummy
    return output

In [65]:
# dataset load
#df = pd.read_csv("weather.csv")
csv_string = input("Input .csv filename: ")
try:
    df = pd.read_csv(csv_string)
except:
    print("File not found.")
    ###quit() (di file Python, pake ini)
print("File loaded successfuly.")
#print(df.head, "\n")

Input .csv filename: weather.csv
File loaded successfuly.


In [66]:
# dataset preprocess

# transform non-numeric data to numeric data
def transform_to_numeric(matrix_data):
    types = matrix_data.dtypes
    labels = matrix_data.columns.values # because pandas select columns using column names
    for i in range(matrix_data.shape[1]):
        type_i = types[i]
        if (type_i == object):
            values = matrix_data[labels[i]].unique()
            dict_i = dict(zip(values, range(len(values)))) # transform every unique object/string into numbers
            matrix_data = matrix_data.replace({labels[i]:dict_i})
    return matrix_data

# previous version's method
#weather_dict = dict([("sunny",0), ("overcast",1), ("rainy",2)])
#label_dict = dict([("no",0), ("yes",1)])
#newdf = df.replace({"outlook":weather_dict})
#newdf = newdf.replace({"play":label_dict})

newdf = transform_to_numeric(df)
#print(newdf.head, "\n")

# scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() #feature_range=(0,1) (min_val, max_val)
scaler.fit(newdf)
data_matrix = scaler.transform(newdf) # di tahap ini, data sudah menjadi numpy array
#print(data_matrix, "\n")

In [70]:
# input and main program

while True:
    hidden_layers = int(input("Input number of hidden layers: "))
    if (hidden_layers <= 10 and hidden_layers >= 0):
        break
    else:
        print("# of hidden layers must be a positive integer and no more than 10.")

nb_nodes = np.empty(hidden_layers)
for i in range(hidden_layers):
    nb_nodes[i] = int(input("Input number of nodes for hidden layer %d : " % i))
    
momentum = 0.01 # manual in-code setting
learning_rate = 0.01 # manual in-code setting

epoch = int(input("Input number of epochs: "))
batch_size = int(input("Input the batch size: "))

result = mini_batch_gradient_descent(data_matrix, hidden_layers, nb_nodes, momentum, learning_rate, epoch, batch_size)

Input number of hidden layers: 3
Input number of nodes for hidden layer 0 : 2
Input number of nodes for hidden layer 1 : 3
Input number of nodes for hidden layer 2 : 4
Input number of epochs: 1
Input the batch size: 1
[2. 3. 4.]
[2. 3. 4. 1.]


In [ ]:
#playground

#print(np.random.randn(10,9) * np.sqrt(1/9))
#x = np.empty((3, 4))
#a = np.zeros( (3,4) )
#b = np.zeros( (2,3) )
#c = np.stack((x,a)) 
#np.concatenate((c, [a]))
#testarr = np.empty((3,4,5))

#emptyarr = list()
#a = np.random.randn(3, 5) * np.sqrt(2/(5+3))
#b = np.random.randn(5, 2) * np.sqrt(2/(5+2))
#emptyarr.append(a)
#emptyarr.append(b)
#print(emptyarr)

# main prog testing
#hidden_layers = 3
#nb_nodes = [3,4,2]

#testarr = initialize_weights(3, [3,4,2], 5)
#for i in range(len(testarr)):
#    print(testarr[i], '\n')